In [7]:
interactive_mode = True
similarities_done = False

In [3]:
if interactive_mode:
    from ipynb.fs.defs.fonctions_preprocess import open_file
    from ipynb.fs.defs.fonctions_preprocess import serialisation_data
    from ipynb.fs.defs.search_humans_referent import *

import spacy
import json

In [4]:
if interactive_mode and not similarities_done:
    data = open_file("lemme_form.json")

In [ ]:
def open_file(file):
    with open(file) as json_data:
        data = json.load(json_data)
    return data

In [3]:
if not similarities_done:
    data = open_file("lemme_form.json")
    print(len(data))

91054


In [4]:
print(type(data))

<class 'dict'>


In [ ]:
i = 100
extract = []
for dic in data:
    if i == 300 :
        break
    extract.append(dic)
    i+=1
    
print(extract)

In [4]:
data = open_file("humans_occ.json")

In [10]:
print(len(data))

23641


In [11]:
def list_into_string(dict_lemme):

    extract = []
    extract_s = ""

    i = 0
    for dic in dict_lemme:
        if i == 700: break
        extract.append(dic)
        i+=1
            
    print("Taille des occurences uniques : ", len(extract))
        
    for word in extract:
        extract_s += word
        extract_s += " "
        
    return extract_s

In [12]:
if not similarities_done:
    corpus_string = list_into_string(data) 
print(len(corpus_string))

Taille des occurences uniques :  700
7163


In [13]:
if not similarities_done:
    #nlp.max_length = 1500000
    nlp = spacy.load("fr_core_news_lg")
    doc1 = nlp(corpus_string)
    doc2 = nlp(corpus_string)

In [14]:
def calculate_similarity (token1, token2):
    """Calculate Similarity between two tokens using SpaCy pre-trained model
    -> Two tokens
    <- similarity score between these two tokens"""

    similarity = token1.similarity(token2)
    
    if similarity != None and similarity >= 0.5 and similarity != 1 : return (token1, token2, similarity) 

In [15]:
def construction_liste_similarity(doc1, doc2):
    """Builds list of similarities. One element of the list is built so : (token1, token2, similarity)
    -> doc1 : list of words
    -> doc2 : other list of words
    <- list of tuples made of two words and their similarities
    """
    
    similarities = []
    
    for i in range (len(doc1)):
        if i%100 == 0 : print ("mot n°", i, "/", len(doc1))
        if doc1[i].has_vector != False: tok = doc1[i]
        for k in range(len(doc2)):
            if doc2[k].has_vector != False : 
                similarity = calculate_similarity(tok, doc2[k])
                
            if similarity != None : similarities.append(similarity)
            #if similarity != None : print(tok, doc2[k], similarity)
                    
    return similarities
    

In [16]:
def extract_unique_referents(doc1, doc2):
    """extract unique_referent using word_similarities
    -> string of words
    <- list of words and dictionnary of similarities
    doc_sim = {word1: [(word, similarity), ..., (word, similarity)], word2 : [(word, similarity), ... (word, similarity)], ... wordn : [(word, similarity), (word, similarity)]}
    
    We use word similarities using spacy module. We take similarities >= 0.5 to delete low similarities and "clean" corpus) 
    """
    
    humans_referent = []
    doc_sim = {}

    similarities = construction_liste_similarity(doc1, doc2)

    print("Similarities are done !")
    
    print(similarities[0:30])
    
    i=0
    for sim in similarities:
        i+=1
        if i%10000 == 0: print("Similarité n°", i, "/", len(similarities) )
        if sim[1] not in humans_referent: humans_referent.append(sim[1])
        if str(sim[0]) not in doc_sim : 
            doc_sim[str(sim[0])] = []
        doc_sim[str(sim[0])].append((str(sim[1]), float(sim[2])))
        

    print("Taille de humans_referent : ", len(humans_referent), " et taille de doc_sim : ", len(doc_sim))
    return (humans_referent, doc_sim)

In [17]:
if not similarities_done:
    humans = extract_unique_referents(doc1, doc2)
    print(len(humans[1]))
    print(type(humans[1]))

mot n° 0 / 700
mot n° 100 / 700
mot n° 200 / 700
mot n° 300 / 700
mot n° 400 / 700
mot n° 500 / 700
mot n° 600 / 700
Similarities are done !
[(non-confiance, l‘organisation, 0.5225424), (non-confiance, parlement, 0.5057618), (non-confiance, insensibilité, 0.63784087), (non-confiance, non-participation, 0.59623164), (non-confiance, co-décision, 0.6465865), (non-confiance, enclenchement, 0.50972414), (non-confiance, linitiative, 0.5300373), (non-confiance, parlementaire, 0.5287417), (non-confiance, partiellement, 0.5242985), (non-confiance, corporellement, 0.5228883), (non-confiance, communalité, 0.5416549), (non-confiance, fondement, 0.5713872), (non-confiance, religiosité, 0.50081515), (non-confiance, compréhensibilité, 0.67381513), (non-confiance, réengagement, 0.6556278), (non-confiance, comportemental, 0.5185476), (non-confiance, cheminement, 0.50257814), (non-confiance, interrelation, 0.5894332), (non-confiance, e-administration, 0.5240783), (non-confiance, pouvoire, 0.5323517), (n

In [ ]:
i = 1
for h in humans[1]:
    if i == 5 : break
    print(h, humans[1][h])
    print()
    i+=1
    

In [18]:
dict_ref = humans[1]
#dict_occ = open_file("dict_occ.json")

In [19]:
def humans_occ (dict_sim):
    """Creates dict of occurences
    -> dict_occ : dictionnary of occcurences of each occurence
    -> dict_human_ref : dictionnary of similarities of each occurence
    <- dictionnary of occurences of most occurence
    """
    
    humans_ref = []
    #humans_occ = {}
    
    
    for dic in dict_sim:
       
        sim_ord = sorted(dict_sim[dic], key=lambda x:x[1], reverse = True)
        ref_max = sim_ord[0]
 
        if ref_max[0] not in humans_ref : humans_ref.append(ref_max[0])
        
        """
        h = str(ref_max[0])
        
        if h not in humans_occ : 
            humans_occ[h]= 0
        humans_occ[h] = dict_occ[h]
        """
    
    print("taille de humans_ref :", len(humans_ref))

    return humans_ref 

In [20]:
fromtest = humans_occ(dict_ref)

taille de humans_ref : 326


In [22]:
print(sorted(fromtest))

['-mise', '-retour', '..', '/enfant', '48h00', 'accidentalité', 'accidente', 'animosité', 'anthropologie', 'applicable', 'argente', 'arithmétique', 'arrivant', 'arrière-fond', 'assignation', 'atteind', 'attendant', 'audition', 'auto-limitation', 'auto-protection', 'auto-représentation', 'auto-surveillance', 'autoroutier', 'avril', 'aéroportuaire', 'bas-peuple', 'beau-parent', 'belgique', 'bien', 'bénévolat', 'bénévole', 'cagette', 'chanteur', 'chaussé', 'cheminement', 'claire', 'co-intervention', 'co-élaboration', 'codéveloppement', 'cofinancement', 'colmatage', 'comité', 'communale', 'communalité', 'communautée', 'commune', 'compensation', 'compréhensibilité', 'compréhensible', 'compte-rendu', 'conceptuel', 'concordat', 'condamnable', 'condescendance', 'conduire', 'confraternité', 'confédération', 'conscience', 'constatation', 'constitutionalité', 'consultant', 'conséquence', 'contestation', 'contractualisation', 'contrairement', 'contravention', 'contre-point', 'contre-proposition', 

In [ ]:
def serialisation_data (data, title):
  """
  Serialize data in a json file
  -> Title mus be a string : title.json
  <- Save a file in desktop
  """

  with open(title, "w+") as file:
    json.dump(data, file)

In [17]:
serialisation_data(fromtest, "humans_occ.json")